In [66]:
import statistics
import yfinance as yt
import numpy as np
import matplotlib.pyplot as plt
import random
import pandas as pd
import yahoo_fin
import yfinance as yt
from yahoo_fin import stock_info as si

from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed
from datetime import date, datetime
indi_ret=[]
indi_risk=[]
port_ret=[]
portfo_risk=[]

In [67]:
class Portfolio():
    def __init__(self, n_companies, weights, ticker):
        self.n_companies = n_companies
        self.weights = weights
        self.ticker = ticker
        self.returns, self.exp_returns, self.risk = self.risk_return_individual()
       
    def risk_return_individual(self):
        risk = []
        exp_returns = []
        returns = []
        for i in range(len(self.ticker)):
            data = yt.download(self.ticker[i])
            ret = []
            lst = data['Close']
            for j in range(len(lst)-1):
                ret.append(((lst[j+1]-lst[j])/lst[j])*100)
            returns.append(ret)
            exp_returns.append(statistics.mean(ret))
            risk.append(statistics.stdev(ret))
        print('\nRisk of the individual companies : ',risk)
        indi_ret.append([exp_returns])
        indi_risk.append([risk])
        return returns, exp_returns, risk
    def portfolio_return(self):
        port_returns = sum([a*b for a,b in zip(self.weights, self.exp_returns)])
        port_ret.append([port_returns])
        print('Portfolio expected return is : ',port_returns)
    def portfolio_risk(self):
        term_1 = sum([w*w*r*r for w, r in zip(self.weights,self.risk)])
        cov_matrix = np.zeros((self.n_companies,self.n_companies))
        for i in range(self.n_companies):
            for j in range(self.n_companies):
                min_len = min(len(self.returns[i]), len(self.returns[j]))
                cov_matrix[i][j] = np.cov(self.returns[i][-min_len:],self.returns[j][-min_len:], ddof = 1)[0][1]
        term_2 = 2*(sum([self.weights[i]*self.weights[j]*cov_matrix[i][j] for i in range(self.n_companies) for j in range(i+1, self.n_companies)]))
        port_risk = (term_1 + term_2)**0.5
        portfo_risk.append([port_risk])
        print('Portfolio risk is :', port_risk)  
ticker=[]



In [68]:

n_com=[]
weightss=[]
get_por=int(input('enter the no of portfolio count: '))
for i in range(get_por):
    n_companies = int(input('Enter the number of companies in your portfolio : '))
    n_com.append(n_companies)
    tc=[]
    ticker.append([tc])
    for i in range(n_companies):
        g=input("enter a company name : ")
        tc.append(g)
#         tc.append(random.choice(com))
    weights = []
    weightss.append(weights)
    for i in range(n_companies):
        weights.append(float(input(f'Enter your weights of the  companies {tc[i]} : ')))
    x = Portfolio(n_companies, weights,tc)
    x.portfolio_return()
    x.portfolio_risk()


# overall=pd.DataFrame({'individual risk':indi_risk,'individual return':indi_ret,'portfolio ret':port_ret,'portfolio risk':portfo_risk,'ticker':ticker})
p=pd.DataFrame({'individual risk':indi_risk})
p1=pd.DataFrame({'individual return':indi_ret})
p2=pd.DataFrame({'portfolio ret':port_ret})
p3=pd.DataFrame({'portfolio risk':portfo_risk})
# print(ticker)
print(p)
print(p1)
print(p2)
print(p3)




# print(overall)
# po=pd.concat(p,p1)
# print(overall)
# po1=pd.concat(p2,p3)

# overall=pd.DataFrame({p,p1,p2,p3})
# print(overall)

enter the no of portfolio count: 1
Enter the number of companies in your portfolio : 2
enter a company name : ibm
enter a company name : goog
Enter your weights of the  companies ibm : 0.8
Enter your weights of the  companies goog : 0.2
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Risk of the individual companies :  [1.591392178861302, 1.9439851175631495]
Portfolio expected return is :  0.045346600645564686
Portfolio risk is : 1.4603275810043541
                             individual risk
0  [[1.591392178861302, 1.9439851175631495]]
                              individual return
0  [[0.031269421373339906, 0.1016553177344638]]
            portfolio ret
0  [0.045346600645564686]
         portfolio risk
0  [1.4603275810043541]


In [69]:
n_com=[]
tc=[]
c=int(input('Enter the number of ticker: '))
for k in range(c):
    g=input("enter a company name : ")
    tc.append(g)
print(tc)
start_date ='2020-01-01'
end_date = '2023-01-01'
for kk in tc:
    st= yt.download(kk,start=start_date,end=end_date)
    stock_data=st.to_csv('stock.csv')
   
# # Load the stock data
stock_data = pd.read_csv('stock.csv') # Replace with your dataset
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from datetime import date
from tensorflow.keras.layers import Input

# Load the stock data
stock_data = pd.read_csv('stock.csv') # Replace with your dataset
stock_data.head()

# Preprocess the data
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(stock_data[['Close']].values)

# Split the data into training and testing sets
train_size = int(len(scaled_data) * 0.8)
train_data = scaled_data[:train_size]
test_data = scaled_data[train_size:]

# Reshape the training data
train_data = np.reshape(train_data, (train_data.shape[0], 1))

# Define the autoencoder model
input_dim = train_data.shape[1]
encoding_dim = 64

autoencoder = Sequential()
autoencoder.add(Dense(encoding_dim, activation='relu', input_shape=(input_dim,)))
autoencoder.add(Dense(input_dim, activation='linear'))

# # # Define the Omega model
autoencoder_input =Input(shape=(input_dim,))
encoded_data = autoencoder(autoencoder_input)

# Define the model
timesteps = 10
lstm_units = 64

model = Sequential()
model.add(LSTM(lstm_units, activation='relu', input_shape=(timesteps, 1)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

# Train the model
model.fit(train_data, train_data, epochs=100)
lstm_input = RepeatVector(timesteps)(encoded_data)
predicted_data = model(lstm_input)

omega_model = Model(autoencoder_input, predicted_data)

# Compile and train the Omega model
omega_model.compile(optimizer='adam', loss='mean_squared_error')
omega_model.fit(train_data[:-timesteps], train_data[timesteps:], epochs=100, batch_size=32)

# Generate predictions on the test data
encoded_test_data = autoencoder.predict(test_data  [:-timesteps])
predicted_test_data = model.predict(np.repeat(encoded_test_data, timesteps, axis=0))
predicted_test_data = np.reshape(predicted_test_data, (-1, 1))

# Scale the predicted data back to the original range
predicted_test_data = scaler.inverse_transform(predicted_test_data)

d=test_actual_data - predicted_test_data
D=d.flatten()
# Evaluate the model
test_actual_data= stock_data['Close'].values[train_size + timesteps:]
mse = np.mean(D**2)
print("Mean Squared Error:",mse)

Enter the number of ticker: 2
enter a company name : IBM
enter a company name : goog
['IBM', 'goog']
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
Epoch 1/100
19/19 [==============================] - 1s 3ms/step - loss: 0.3338
Epoch 2/100
19/19 [==============================] - 0s 3ms/step - loss: 0.2704
Epoch 3/100
19/19 [==============================] - 0s 2ms/step - loss: 0.2110
Epoch 4/100
19/19 [==============================] - 0s 2ms/step - loss: 0.1538
Epoch 5/100
19/19 [==============================] - 0s 2ms/step - loss: 0.1044
Epoch 6/100
19/19 [==============================] - 0s 2ms/step - loss: 0.0649
Epoch 7/100
19/19 [==============================] - 0s 2ms/step - loss: 0.0416
Epoch 8/100
19/19 [==============================] - 0s 2ms/step - loss: 0.0298
Epoch 9/100
19/19 [==============================] - 0s 2ms/step - loss: 0.0254
Epoch 10/100
19/19 [=====================

19/19 [==============================] - 0s 2ms/step - loss: 1.7463e-04
Epoch 97/100
19/19 [==============================] - 0s 2ms/step - loss: 1.7233e-04
Epoch 98/100
19/19 [==============================] - 0s 2ms/step - loss: 1.7248e-04
Epoch 99/100
19/19 [==============================] - 0s 2ms/step - loss: 1.6964e-04
Epoch 100/100
19/19 [==============================] - 0s 2ms/step - loss: 1.6808e-04
Epoch 1/100
19/19 [==============================] - 2s 8ms/step - loss: 0.2327
Epoch 2/100
19/19 [==============================] - 0s 8ms/step - loss: 0.0703
Epoch 3/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0335
Epoch 4/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0120
Epoch 5/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0048
Epoch 6/100
19/19 [==============================] - 0s 8ms/step - loss: 0.0042
Epoch 7/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0038
Epoch 8/100
19/19 [========

19/19 [==============================] - 0s 7ms/step - loss: 0.0031
Epoch 98/100
19/19 [==============================] - 0s 7ms/step - loss: 0.0032
Epoch 99/100
19/19 [==============================] - 0s 8ms/step - loss: 0.0031
Epoch 100/100
45/45 [==============================] - 0s 2ms/step
Mean Squared Error: 748.5039511760556


In [75]:
a=(0.5,0.5)
x=Portfolio(c,a,tc)
x.portfolio_return()
x.portfolio_risk()

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed

Risk of the individual companies :  [1.591392178861302, 1.9439851175631495]
Portfolio expected return is :  0.06646236955390186
Portfolio risk is : 1.463318183630116
